In [1]:
import os
import pandas as pd
import warnings 
import numpy as np
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt

warnings.filterwarnings(action='ignore')
rc('font',family='AppleGothic')
rcParams['axes.unicode_minus']=False

In [2]:
path = './data_전용면적별/'

train = pd.read_csv(path+'train_new.csv')
test = pd.read_csv(path+'test_new.csv')

train.shape, test.shape

((1547, 70), (567, 68))

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import cross_val_score



In [18]:

sel = ['임대료','단지내주차면수','전용면적별세대수', '면적_세대수_곱', '면적_세대수_곱_총합','면적_세대수_비율',
      '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시',
       '서울특별시', '세종특별자치시', '울산광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도',
      '나머지', '분양상가', '분양아파트']

X = train[sel]
y = train[['전용면적_비례_차량수']]

# 피처스케일링
scaler = MinMaxScaler()# MinMaxScaler StandardScaler
X_nor = scaler.fit_transform(X)
tmp_X_nor = X_nor[:,:6]
X_poly = PolynomialFeatures(degree=2, include_bias=False).fit_transform(tmp_X_nor)  

X_all = np.concatenate((X_poly, X_nor[:,6:]), axis=1)

# 모델 
rf = RandomForestRegressor()

# 회귀는 교차검증
rf_scores = cross_val_score(rf, X_all, y,scoring='neg_mean_absolute_error', cv=5)
rf_score = np.abs(rf_scores.mean())
print("RandomForestRegressor : ", rf_score)


RandomForestRegressor :  34.27558901236473


In [5]:
X_test = test[sel]

X_test_nor = scaler.transform(X_test)
tmp_X_test_nor = X_test_nor[:,:6]
X_test_poly = PolynomialFeatures(degree=2, include_bias=False).fit_transform(tmp_X_test_nor)
X_test_all = np.concatenate((X_test_poly, X_test_nor[:,6:]), axis=1)

rf.fit(X_all, y)
pred = rf.predict(X_test_all)
print(X_test.shape)
print(pred.shape)

(567, 25)
(567,)


In [6]:
test['전용면적_비례_차량수'] = pred
test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,O,ERROR1_lb,나머지,분양상가,분양아파트,전용면적별세대수,면적_세대수_곱,면적_세대수_곱_총합,면적_세대수_비율,전용면적_비례_차량수
0,C1003,480,아파트,경상남도,행복주택,16,29.0,J,12000000.0,61000.0,...,0,0,1,0,0,128,2048,12492,0.163945,48.917121
1,C1003,480,아파트,경상남도,행복주택,26,29.0,J,18800000.0,96000.0,...,0,0,1,0,0,30,780,12492,0.062440,20.654660
2,C1003,480,아파트,경상남도,행복주택,26,29.0,J,19600000.0,100000.0,...,0,0,1,0,0,220,5720,12492,0.457893,137.171517
3,C1003,480,아파트,경상남도,행복주택,36,29.0,J,25600000.0,131000.0,...,0,0,1,0,0,68,2448,12492,0.195965,47.980987
4,C1003,480,아파트,경상남도,행복주택,44,29.0,J,30000000.0,154000.0,...,0,0,1,0,0,34,1496,12492,0.119757,38.270040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,C2688,150,아파트,전라남도,국민임대,46,1.0,H,16448000.0,194290.0,...,0,0,1,0,0,16,736,4552,0.161687,18.311501
563,C2688,150,아파트,전라남도,영구임대,24,1.0,C,9113000.0,86450.0,...,0,0,1,0,0,26,624,4552,0.137083,16.850198
564,C2688,150,아파트,전라남도,영구임대,24,1.0,C,9163000.0,86940.0,...,0,0,1,0,0,4,96,4552,0.021090,1.498967
565,C2691,765,아파트,경기도,공공임대(10년),74,25.0,A,66000000.0,550000.0,...,0,0,1,0,0,175,12950,62510,0.207167,225.922148


In [7]:
final = test.groupby("단지코드", as_index=False).agg({'전용면적_비례_차량수': 'sum'})
final.columns = ['code', 'num']
final

,code,num
0,C1003,292.994325
1,C1006,463.987445
2,C1016,644.722282
3,C1019,268.314447
4,C1030,29.693026
...,...,...
142,C2646,558.247721
143,C2653,990.064102
144,C2676,347.144380
145,C2688,117.134691


In [8]:
err = {'code':['C2675', 'C2335', 'C1327'],
           'num':['0', '0', '0']}
err_df = pd.DataFrame(err)

final = pd.concat([final, err_df])
final.reset_index(drop=True, inplace=True)
final

,code,num
0,C1003,292.994
1,C1006,463.987
2,C1016,644.722
3,C1019,268.314
4,C1030,29.693
...,...,...
145,C2688,117.135
146,C2691,1119.22
147,C2675,0
148,C2335,0


In [9]:
PATH = './sub/'

final.to_csv(PATH + "0727_1st_sub.csv", index=False)
os.listdir(PATH)

['0727_1st_sub.csv']

> 결과 : 108.0523610055